In [1]:
import essentia
from essentia.standard import *
import IPython
from scipy.io import wavfile
import matplotlib.pyplot as plt

[   INFO   ] MusicExtractorSVM: no classifier models were configured by default


In [ ]:
# Loading audio file in mono
audio = MonoLoader(filename='../loops/audio2.wav')() 

# Phase 1: compute the onset detection function
#hfc method
od1 = OnsetDetection(method='hfc')

# Let's also get the other algorithms we will need, and a pool to store the results
w = Windowing(type = 'hann')
fft = FFT() # this gives us a complex FFT
c2p = CartesianToPolar() # and this turns it into a pair (magnitude, phase)
pool = essentia.Pool()

# Computing onset detection functions.
for frame in FrameGenerator(audio, frameSize = 1024, hopSize = 512):
    mag, phase, = c2p(fft(w(frame)))
    pool.add('features.hfc', od1(mag, phase))

# Phase 2: compute the actual onsets locations
onsets = Onsets()

onsets_hfc = onsets(# this algo expects a matrix, not a vector
                    essentia.array([ pool['features.hfc'] ]),

                    # you need to specify weights, but as there is only a single
                    # function, it doesn't actually matter which weight you give it
                    [ 1 ])



# Mark onsets on the audio, which we'll write back to disk
# We use beeps instead of white noise and stereo signal as it's more distinctive

#silence = [0.] * len(audio)

#beeps_hfc = AudioOnsetsMarker(onsets=onsets_hfc, type='beep')(silence)
#AudioWriter(filename='audios/audio1_onsets_hfc.mp3', format='mp3')(StereoMuxer()(audio, beeps_hfc))

In [ ]:
plt.plot(audio)
for onset in onsets_hfc:
    plt.axvline(x=onset*44100, color='red')

plt.title("Audio waveform and the estimated onset positions (HFC onset detection function)")
plt.show()

IPython.display.Audio(audio, rate=44100)


In [ ]:

    
audio_test = audio[0:12500]    
IPython.display.Audio(chopList[0],rate = 44100)

print(chopList[0])


In [ ]:
IPython.display.Audio(chopList[1],rate = 44100)


In [ ]:
IPython.display.Audio(chopList[2],rate = 44100)

In [ ]:
IPython.display.Audio(chopList[3],rate = 44100)


In [ ]:
IPython.display.Audio(chopList[4],rate = 44100)

In [ ]:
def get_onsets_hfc(audio,fs):
    od = OnsetDetection(method='hfc')

    w = Windowing(type='hann')
    fft = FFT()  # this gives us a complex FFT
    c2p = CartesianToPolar()  # and this turns it into a pair (magnitude, phase)
    pool = essentia.Pool()

    # Computing onset detection functions.
    for frame in FrameGenerator(audio, frameSize=1024, hopSize=512):
        mag, phase, = c2p(fft(w(frame)))
        pool.add('features.hfc', od(mag, phase))

        onsets = Onsets()
        onsets_hfc_seconds = onsets(essentia.array([pool['features.hfc']]), [1])

    onsets_hfc_samples = onsets_hfc_seconds * fs #retun
    return onsets_hfc_samples, onsets_hfc_seconds


def get_slices(audio, fs):
    onsets_hfc = get_onsets_hfc(audio,fs)[0]
    chopList = []
    for onset in range(len(onsets_hfc) - 1):
        chopList.append(audio[int(onsets_hfc[onset]):int((onsets_hfc[onset + 1]))])

    return chopList



In [ ]:
audio = MonoLoader(filename='../loops/audio2.wav')
chops=get_slices(audio,44100)

In [5]:
audio = MonoLoader(filename='../loops/audio2.wav')
chops=get_slices(audio,44100)

TypeError: Error cannot convert argument UNDEFINED to VECTOR_REAL